In [1]:
import io
from ipyfileupload.widgets import DirectoryUploadWidget
def _upload():

    _upload_widget = DirectoryUploadWidget()

    def _cb(change):
        for name, file in change['owner'].files.items():
            decoded = io.StringIO(file.decode('utf-8'))
            print('Uploaded `{}` ({:.2f} kB)'.format(
                name, len(decoded.read()) / 2 **10))

    _upload_widget.observe(_cb, names='base64_files')
    display(_upload_widget)
    return _upload_widget

widget = _upload()

DirectoryUploadWidget(label='Browse', _dom_classes=('widget_item', 'btn-group'))

In [2]:
from classes.ipywidgets import *
from classes.Titration import Titration, TitrationCLI
from ipyfileupload.widgets import DirectoryUploadWidget

TitrationWidget.titration = Titration()
DirectoryUploadWidget()

DirectoryUploadWidget(label='Browse', _dom_classes=('widget_item', 'btn-group'))

In [3]:


view = TitrationFilesView()
upload = TitrationDirUploader()
upload.add_observer(view)
display(
    upload,
    view
)

TitrationDirUploader(label='Upload titration directory...', style=ButtonStyle(), _dom_classes=('widget_item', 'btn-group'))

TitrationFilesView(children=(VBox(children=(Label(value='Data files', _dom_classes=('panel-header-title',)),), _dom_classes=('panel-heading',)), VBox(_dom_classes=('panel-body',))), layout=Layout(width='250px'), _dom_classes=('panel', 'panel-default', 'data-files-view'))

In [4]:
startParams = StartParamContainer()
protocole = ProtocoleContainer()

upload.add_observer(startParams)
upload.add_observer(protocole)

startParams.add_observer(protocole.protocole)

display(startParams, protocole)

StartParamContainer(children=(VBox(children=(NameWidget(value='Unnamed Titration', description='Experiment name:', layout=Layout(width='335px'), style=DescriptionStyle(description_width='130px'), _dom_classes=('panel-header-title',)),), _dom_classes=('panel-heading',)), VBox(children=(MoleculesContainer(children=(SingleMolContainer(children=(Label(value='Analyte', layout=Layout(left='150px')), MoleculeNameWidget(value='analyte', description='Name:', layout=Layout(width='230px'), style=DescriptionStyle(description_width='130px')), ConcentrationWidget(value=0.0, description='Concentration (µM):', layout=Layout(width='230px'), max=100000.0, style=DescriptionStyle(description_width='130px')))), SingleMolContainer(children=(Label(value='Titrant'), MoleculeNameWidget(value='titrant', layout=Layout(width='100px')), ConcentrationWidget(value=0.0, layout=Layout(width='100px'), max=100000.0))))), FloatControlWidget(value=0.0, description='Analyte volume (µL):', layout=Layout(width='230px'), max=100000.0, style=DescriptionStyle(description_width='130px')), FloatControlWidget(value=0.0, description='Total volume (µL):', layout=Layout(width='230px'), max=100000.0, style=DescriptionStyle(description_width='130px'))), _dom_classes=('panel-body',))), layout=Layout(width='400px'), _dom_classes=('panel', 'panel-default'))

ProtocoleContainer(children=(VolumePanel(children=(VBox(children=(Label(value='Added titrant volumes', _dom_classes=('panel-header-title',)), HBox(children=(Button(button_style='primary', description='Add', icon='plus-circle', layout=Layout(width='50%'), style=ButtonStyle()), Button(button_style='warning', description='Remove', disabled=True, icon='minus-circle', layout=Layout(width='50%'), style=ButtonStyle())), layout=Layout(padding='5px 5px 5px 5px'))), _dom_classes=('panel-heading',)), VBox(children=(VBox(children=(VolumeWidget(value=0.0, description='Step 0', disabled=True, layout=Layout(height='28px', width='100%'), max=100000.0, style=DescriptionStyle(description_width='50%')),), layout=Layout(display='flex', flex_flow='column', padding='5px 5px 5px 5px')),), _dom_classes=('panel-body',)), VBox(children=(Button(button_style='success', description='Validate', icon='check', layout=Layout(width='100%'), style=ButtonStyle()),), _dom_classes=('panel-footer',))), layout=Layout(height='auto', width='250px'), _dom_classes=('panel', 'panel-default', 'volumes-container')), ProtocolePanel(children=(VBox(children=(Label(value='Protocole', _dom_classes=('panel-header-title',)),), _dom_classes=('panel-heading',)), VBox(children=(HTML(value='<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Step</th>\n      <th>Added titrant (µL)</th>\n      <th>Total titrant (µL)</th>\n      <th>Total volume (µL)</th>\n      <th>[titrant] (µM)</th>\n      <th>[analyte] (µM)</th>\n      <th>[titrant]/[analyte]</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>0</td>\n      <td>0</td>\n      <td>0</td>\n      <td>0</td>\n      <td>NaN</td>\n      <td>NaN</td>\n      <td>NaN</td>\n    </tr>\n  </tbody>\n</table>', _dom_classes=('rendered_html', 'protocole-table')),), _dom_classes=('panel-body',))), _dom_classes=('panel', 'panel-default'))), layout=Layout(justify_content='space-around'))

In [5]:
import bqplot
import math 
from itertools import chain
from ipywidgets import FloatSlider, IntSlider, ToggleButton
import classes
import importlib

importlib.reload(classes)
from classes.bqplotwidgets import IntensityPlot
#HBox([CutoffSlider(),IntensityBarPlot()])

class IntensityPanel(TitrationWidget, PanelContainer):
    def __init__(self, *args, **kwargs):
        PanelContainer.__init__(self, *args, **kwargs)

        #self.heading=HBox()
        #self.heading.add_class("panel-heading")

        self.add_class('intensity-panel')

        self.label = Label("Chemical shift intensities")
        self.label.add_class('panel-header-title')

        self.plot_widgets = IntensityPlot()

        self.toolbar = bqplot.Toolbar(figure= self.plot_widgets.plot)
        self.toolbar.add_class('barplot-toolbar')

        self.set_heading([self.label])

        self.set_content([self.plot_widgets, self.toolbar])
        self.children = (self.heading, self.content)
        
plotc = IntensityPanel()
display(plotc)
plotc.plot_widgets.plot.save_png('./akanr.png')

IntensityPanel(children=(VBox(children=(Label(value='Chemical shift intensities', _dom_classes=('panel-header-title',)),), _dom_classes=('panel-heading',)), VBox(children=(IntensityPlot(children=(IntSlider(value=10, description='Step', layout=Layout(height='95%', padding='30px 0px 30px 0px'), max=10, min=1, orientation='vertical'), CutoffSlider(value=0.1, description='Cut-off', layout=Layout(height='95%', padding='30px 0px 30px 0px'), max=1.7487105999564347, orientation='vertical', step=0.01), IntensityBarPlot(axes=[Axis(grid_lines='none', label='Residues', num_ticks=11, scale=OrdinalScale()), Axis(grid_color='#D1D0CE', grid_lines='dashed', orientation='vertical', scale=LinearScale(max=1.7487105999564347, min=0.0), tick_format='0.2f')], fig_margin={'bottom': 60, 'right': 60, 'left': 60, 'top': 60}, layout=Layout(height='95%', width='100%'), marks=[Bars(colors=['#38ACEC'], interactions={'click': 'select', 'hover': 'tooltip', 'legend_hover': 'highlight_axes'}, scales={'x': OrdinalScale(), 'y': LinearScale(max=1.7487105999564347, min=0.0)}, scales_metadata={'x': {'dimension': 'x', 'orientation': 'horizontal'}, 'color': {'dimension': 'color'}, 'y': {'dimension': 'y', 'orientation': 'vertical'}}, selected=[0, 1, 2, 3, 4, 7, 8, 39, 41, 67, 68, 70, 73, 74, 75, 76, 80, 82, 85, 87, 91, 92, 94, 95, 99, 100], selected_style={'fill': 'orange', 'stroke': 'white'}, stroke='#FFFFFF', tooltip=Tooltip(fields=['x', 'y'], formats=['', '.2f'], labels=['Residue', 'Intensity']), tooltip_style={'opacity': 0.9}, x=array([166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178,
       179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191,
       192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205,
       206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218,
       219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231,
       232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244,
       245, 246, 247, 248, 249, 250, 251, 253, 254, 255, 256, 257, 258,
       260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272]), y=array([  3.96001263e-01,   1.34078335e-01,   2.06011747e-01,
         4.51144057e-01,   1.04000769e-01,   2.90006896e-02,
         6.79411510e-03,   1.07026913e-01,   1.04000769e-01,
         5.00399840e-03,   4.40549657e-02,   5.01022954e-02,
         6.86466314e-02,   4.40549657e-02,   5.01597448e-03,
         1.90168346e-02,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   1.14017543e-02,
         0.00000000e+00,   1.14017543e-02,   2.00000000e-03,
         1.50119952e-02,   2.40000000e-02,   5.00399840e-03,
         1.00000000e-03,   6.01331855e-03,   2.90006896e-02,
         3.40052937e-02,   2.03960781e-03,   4.30074412e-02,
         2.40000000e-02,   5.03587132e-03,   3.02654919e-03,
         2.90006896e-02,   2.60007692e-02,   1.50119952e-02,
         2.40024082e-01,   1.50119952e-02,   1.79069819e-01,
         7.85875308e-03,   0.00000000e+00,   3.40052937e-02,
         9.10562464e-02,   1.70011764e-02,   0.00000000e+00,
         6.80073525e-02,   2.03960781e-03,   0.00000000e+00,
         0.00000000e+00,   7.50095994e-02,   9.00888450e-03,
         8.00249961e-03,   4.50111097e-02,   0.00000000e+00,
         0.00000000e+00,   2.40000000e-02,   3.60005556e-02,
         2.40008333e-02,   5.80675469e-02,   1.14017543e-02,
         6.10209800e-02,   5.70014035e-02,   2.23606798e-02,
         9.90202000e-02,   5.08566613e-01,   4.90467084e-01,
         3.70345784e-02,   5.12516692e-01,   8.21671467e-02,
         5.15270803e-02,   1.58579083e+00,   1.95473783e-01,
         3.26675925e-01,   3.61354397e-01,   7.62758153e-02,
         9.70000000e-02,   2.44131112e-03,   1.29000155e-01,
         4.35343543e-02,   1.12171476e-01,   6.50249183e-02,
         5.50058179e-02,   1.17008376e-01,   6.00962561e-02,
         1.33005413e-01,   8.59883713e-02,   4.61406545e-02,
         9.10711810e-02,   1.13553688e-01,   1.14092769e-01,
         1.14

In [6]:
from classes.bqplotwidgets import ShiftMap
        
fig = ShiftMap()
display(fig)

ShiftMap(axes=[Axis(grid_color='#D1D0CE', grid_lines='dashed', label='Proton chemical shifts', num_ticks=10, scale=LinearScale(max=130.77682, min=110.51568), tick_format='0.2f'), Axis(grid_color='#D1D0CE', grid_lines='dashed', label='Azote chemical shifts', num_ticks=10, orientation='vertical', scale=LinearScale(max=9.68694, min=6.80022), tick_format='0.2f'), ColorAxis(label='Step', offset={'value': 0, 'scale': LinearScale(max=130.77682, min=110.51568)}, orientation='vertical', scale=ColorScale(scheme='Purples'), side='right')], fig_margin={'bottom': 60, 'right': 60, 'left': 60, 'top': 60}, layout=Layout(height='width', width='100%'), marks=[Scatter(color=array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,
        6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,
        1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,
        7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,
        2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,
        8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,
        3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,
        9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,
        4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
       10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,
        5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,
        0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,
        6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,
        1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,
        7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,
        2,  3,  4,  5,  6,  7,  8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,
        8,  9, 10,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), colors=['DeepSkyBlue'], display_names=False, interactions={'click': 'select', 'hover': 'tooltip', 'legend_hover': 'highlight_axes'}, names=array(['256', '256', '256', '256', '256', '256', '256', '256', '256',
       '256', '256', '257', '257', '257', '257', '257', '257', '257',
       '257', '257', '257', '257', '258', '258', '258', '258', '258',
       '258', '258', '258', '258', '258', '258', '260', '260', '260',
       '260', '260', '260', '260', '260', '260', '260', '260', '261',
       '261', '261', '261', '261', '261', '261', '261', '261', '261',
       '261', '264', '264', '264', '264', '264', '264', '264', '264',
       '264', '264', '264', '265', '265', '265', '265', '265', '265',
       '265', '265', '265', '265', '265', '218', '218', '218', '218',
       '218', '218', '218', '218', '218', '218', '218', '219', '219',
       '219', '219', '219', '219', '219', '219', '219', '219', '219',
       '221', '221', '221', '221', '221', '221', '221', '221', '221',
       '221', '221', '224', '224', '224', '224', '224', '224', '224',
       '224', '224', '224', '224', '225', '225', '225', '225', '225',
       '225', '225', '225', '225', '225', '225', '226', '226', '226',
       '226', '226', '226', '226', '226', '226', '226', '226', '227',
       '227', '227', '227', '227', '227', '227', '227', '227', '227',
       '227', '231', '231', '231', '231', '231', '231', '231', '231',
       '231', '231', '231', '233', '233', '233', '233', '233', '233',
       '233', '233', '233', '233', '233', '236', '236', '236', '236',
       '236', '236', '236', '236', '236', '236', '236', '238', '238',
       '238', '238', '238', '238', '238', '238', '238', '238', '238',
       '242', '242', '242', '242', '242', '242', '242', '242', '242',
       '242', '242', '243', '243', '243', '243', '243', '243', '243',
       '243', '243', '243', '243', '245', '245', '245', '245', '245',
       '245', '245', '245', '245', '245', '245', '246', '246', '246',
       '246', '246', '246', '246', '246', '246', '246', '246', '250',
       '250', '250', '250', '250', '250', 

In [17]:
print(TitrationWidget.titration.filtered)

{256: (256, [112.069, 112.035, 111.903, 111.867, 111.705, 111.676, 111.709, 111.692, 111.632, 111.645, 111.673], [8.999, 8.999, 9.0, 9.0, 9.003, 9.002, 9.004, 9.005, 9.003, 9.004, 9.004]), 257: (257, [120.767, 120.768, 120.768, 120.748, 120.663, 120.681, 120.63, 120.652, 120.638, 120.629, 120.643], [9.24, 9.251, 9.251, 9.359, 9.456, 9.468, 9.48, 9.485, 9.49, 9.497, 9.495]), 258: (258, [116.138, 116.137, 116.193, 116.222, 116.3, 116.302, 116.284, 116.27, 116.319, 116.328, 116.344], [9.19, 9.19, 9.196, 9.197, 9.199, 9.199, 9.202, 9.202, 9.198, 9.198, 9.201]), 260: (260, [121.784, 121.765, 121.709, 121.689, 121.368, 121.386, 121.372, 121.331, 121.321, 121.311, 121.333], [7.747, 7.749, 7.757, 7.757, 7.798, 7.799, 7.801, 7.802, 7.804, 7.803, 7.804]), 261: (261, [116.119, 116.112, 116.162, 116.145, 116.199, 116.221, 116.211, 116.229, 116.198, 116.254, 116.223], [8.245, 8.246, 8.247, 8.248, 8.249, 8.249, 8.247, 8.248, 8.248, 8.246, 8.247]), 264: (264, [112.369, 112.377, 112.337, 112.34, 112.2